In [3]:
from time import sleep
from datetime import datetime
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import pickle
import ast
import shap
import matplotlib.pyplot as plt
from hypermapper import optimizer
from joblib import dump, load
%matplotlib inline

In [4]:
df = pd.read_csv("../serv_rec_full_nprint_10pkts.csv")

In [5]:
df['filename'] = df['filename'].str.extract(r'\.pcap_(\d+\.\d+\.\d+\.\d+)')
df['ip_label'] = df['filename'].str.extract(r'^(\d+\.\d+)')
df['ip_label'] = df['ip_label'].astype('category').cat.codes
df.drop('filename', axis=1, inplace=True)
df

,pkt_0_ipv4_tos_0,pkt_0_ipv4_tos_1,pkt_0_ipv4_tos_2,pkt_0_ipv4_tos_3,pkt_0_ipv4_tos_4,pkt_0_ipv4_tos_5,pkt_0_ipv4_tos_6,pkt_0_ipv4_tl_5,pkt_0_ipv4_tl_6,pkt_0_ipv4_tl_7,...,pkt_9_udp_cksum_8,pkt_9_udp_cksum_9,pkt_9_udp_cksum_10,pkt_9_udp_cksum_11,pkt_9_udp_cksum_12,pkt_9_udp_cksum_13,pkt_9_udp_cksum_14,pkt_9_udp_cksum_15,Label,ip_label
0,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,twitch,46
1,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,twitch,88
2,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,twitch,46
3,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,twitch,88
4,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,twitch,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58084,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,zoom,88
58085,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,zoom,88
58086,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,zoom,88
58087,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,zoom,88


In [6]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_X_train = df_train.iloc[:, :-2]
df_Y_train = df_train.iloc[:, -2:]
df_X_test = df_test.iloc[:, :-2]
df_Y_test = df_test.iloc[:, -2:]

In [7]:
selected_columns = [ 242,  240,  168, 1655,  172, 1999,  990, 1325,  659,  331]
print(len(selected_columns))
df_X_train = df_X_train.iloc[:, selected_columns]
df_X_test = df_X_test.iloc[:, selected_columns]

10


In [8]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from joblib import dump, load
def get_accuracy_for_feat_subset(df_X_train, df_X_test, df_Y_train, df_Y_test, feat_idx, custom_text_ext):
    print(feat_idx)
    if not feat_idx:
        return 1, 0.00001
    new_train = df_X_train.iloc[:, feat_idx]
    new_test = df_X_test.iloc[:, feat_idx]
    threat_model = RandomForestClassifier(random_state=42)
    threat_model.fit(new_train, df_Y_train.iloc[:, 1].to_numpy())
    source_model = RandomForestClassifier(random_state=42)
    source_model.fit(new_train, df_Y_train.iloc[:, 0].to_numpy())
    threat_pred_new = threat_model.predict(new_test)
    source_pred_new = source_model.predict(new_test)
    threat_score = accuracy_score(df_Y_test.iloc[:, 1].to_numpy(), threat_pred_new)  # Source IP re-id classifier (Threat)
    source_score = accuracy_score(df_Y_test.iloc[:, 0].to_numpy(), source_pred_new) # Network Intrusion classifier
    return threat_score, source_score

In [ ]:
parameters_file = "hypermapper-param-serv_rec_full_nprint_10pkts.json"
def bayesian_opt(X):
    feat_idx = []
    for i in range(1, 11):
        if X[f'f{i}'] == 'true':
            feat_idx.append(i-1)
    threat_score, source_score = get_accuracy_for_feat_subset(df_X_train, df_X_test, df_Y_train, df_Y_test, feat_idx, 'bayesian')
    print("Source accuracy and threat accuracy:", source_score, threat_score)
    return threat_score/source_score
value_at_optimum = optimizer.optimize(parameters_file, bayesian_opt)

Design of experiment phase, number of new doe samples = 1024 .......
[0, 1, 4, 5]
Source accuracy and threat accuracy: 0.42528834567051127 0.5149767601996902
[]
Source accuracy and threat accuracy: 1e-05 1
[9]
Source accuracy and threat accuracy: 0.375968325012911 0.4065243587536581
[8]
Source accuracy and threat accuracy: 0.3534171113788948 0.43776897917025304
[8, 9]
Source accuracy and threat accuracy: 0.5689447409192632 0.5771217076949561
[7]
Source accuracy and threat accuracy: 0.3198485109313135 0.4736615596488208
[7, 9]
Source accuracy and threat accuracy: 0.5484592873127905 0.5525047340333965
[7, 8]
Source accuracy and threat accuracy: 0.5071440867619211 0.5546565673954209
[7, 8, 9]
Source accuracy and threat accuracy: 0.6717163022895507 0.651230848683078
[6]
Source accuracy and threat accuracy: 0.30986400413152004 0.46987433293165776
[6, 9]
Source accuracy and threat accuracy: 0.5661903942158719 0.600705801342744
[6, 8]
Source accuracy and threat accuracy: 0.5234980203133069 0.